In [1]:
!rm -rf ./sample_data

### Установка зависимотсей

> для построения маршрутов

In [2]:
%pip install osmnx scikit-learn folium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 2.1 MB/s eta 0:00:00


### Строим маршрут

In [73]:
import osmnx as ox
import networkx as nx

ox.settings.log_console=True

### Данные точек

In [74]:
place = "Nizhniy Novgorod"
mode = "drive"
optimizer = "time"

points = {"выбоина": ["red", [56.329088, 43.987066], [56.329352, 43.992098], [56.330045, 43.995889], [56.331728, 44.002974]],
          "продольная трещина": ["orange", [56.329363, 43.989375], [56.330422, 43.998814], [56.329794, 43.994940], [56.327151, 43.983423]],
          "боковая трещина": ["blue", [56.327862, 43.987735], [56.325003, 43.992044]],
          "трещина аллигатора": ["darkpurple", [56.327049, 43.983050], [56.329417, 43.992244], [56.328479, 43.995958], [56.327067, 43.983131]],
          "продольная трещина": ["pink", [56.325156, 43.979005], [56.330366, 43.991284]],
          "колея": ["black", [56.331549, 43.999298], [56.326559, 43.980067]],
          "ухаб": ["darkred", [56.326314, 43.976423], [56.324948, 43.976893], [56.331389, 43.998594]],
}

### Получаем граф

In [77]:
graph = ox.graph_from_place(place, network_type=mode)

In [78]:
import folium
import random

m = folium.Map((56.328234, 43.983600), tiles="cartodb positron")

### Визуализация точек

In [79]:
for point, coord in points.items():
  for i in range(1, len(coord)):
    folium.Marker(location = (coord[i][0], coord[i][1]),
                  popup = f'тип: {point}\n\nкол-во: {random.randint(1, 10)}',
                  icon = folium.Icon(color=coord[0])).add_to(m)

In [110]:
m

### Строим маршрут

In [342]:
def get_roadmap_points(user_answer):
  points_list = []

  for point in user_answer:
    for data in sorted(points[point][1:]):
      points_list.append(tuple(data))

  return points_list

In [343]:
points_data = [key for key in points]
print(points_data)

random.shuffle(points_data)
shuff_points = points_data[:2]
shuff_points

['выбоина', 'продольная трещина', 'боковая трещина', 'трещина аллигатора', 'колея', 'ухаб']


['колея', 'продольная трещина']

In [344]:
data_user_points = get_roadmap_points(shuff_points)
data_user_points

[(56.326559, 43.980067),
 (56.331549, 43.999298),
 (56.325156, 43.979005),
 (56.330366, 43.991284)]

In [345]:
def get_roadmap(user_points):
  print(user_points)
  for i in range (len(user_points) - 1):
    orig_node = ox.distance.nearest_nodes(graph, user_points[i][1], user_points[i][0])
    dest_node = ox.distance.nearest_nodes(graph, user_points[i + 1][1], user_points[i + 1][0])

    shortest_route = nx.shortest_path(graph, orig_node, dest_node, weight=optimizer)
    shortest_route_map = ox.plot_route_folium(graph, shortest_route)


  return shortest_route_map

In [346]:
data_user_points

[(56.326559, 43.980067),
 (56.331549, 43.999298),
 (56.325156, 43.979005),
 (56.330366, 43.991284)]

In [347]:
shortest_route_map = get_roadmap(data_user_points)

[(56.326559, 43.980067), (56.331549, 43.999298), (56.325156, 43.979005), (56.330366, 43.991284)]


<ipython-input-345-f96c99853795>:8: UserWarning: The `folium` module has been deprecated and will be removed in a future release. You can generate and explore interactive web maps of graph nodes, edges, and/or routes automatically using GeoPandas.GeoDataFrame.explore instead, for example like: `ox.graph_to_gdfs(G, nodes=False).explore()`. See the OSMnx examples gallery for complete details and demonstrations.
  shortest_route_map = ox.plot_route_folium(graph, shortest_route)
<ipython-input-345-f96c99853795>:8: UserWarning: The `folium` module has been deprecated and will be removed in a future release. You can generate and explore interactive web maps of graph nodes, edges, and/or routes automatically using GeoPandas.GeoDataFrame.explore instead, for example like: `ox.graph_to_gdfs(G, nodes=False).explore()`. See the OSMnx examples gallery for complete details and demonstrations.
  shortest_route_map = ox.plot_route_folium(graph, shortest_route)
<ipython-input-345-f96c99853795>:8: User

In [348]:
shortest_route_map

In [349]:
for p in shuff_points:
  for i in range(1, len(points[p])):
    folium.Marker(location = (points[p][i][0],points[p][i][1]),
                  popup = f'тип: {p}\n\nкол-во: {random.randint(1, 10)}',
                  icon = folium.Icon(color=points[p][0])).add_to(shortest_route_map)

In [350]:
shortest_route_map